# Word2Vec v2.5: "Mistake Not"

### Connect to Database

In [1]:
! pip3 install psycopg2-binary --user
import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

    100% |████████████████████████████████| 2.9MB 14.0MB/s ta 0:00:01
You are using pip version 19.0.2, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# connect to database
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(),
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)
# Enter database password below and press Enter.

 ················


In [3]:
# create cursor that is used throughout
try:
    c = connection.cursor()
    print("Connected!")
except:
    print("Connection problem chief!")

Connected!


### Prepare data and train.
1. Get the list of reviewers whose reviews we want (about 17k)
2. Get the dataframe of reviewers, movie IDs with positive reviews
3. Inner join the above two dataframes.
4. Run the list constructor on the join table to construct the training data.
    - Training data is of this format: [['movieid1', 'movieid2', ...], ...]
5. Train Word2Vec on the list of watch histories (which are themselves lists of movie IDs).
6. Save the model.

In [229]:
# Get reviewers with at least 10 positive reviews (rating 7-10 inclusive)
c.execute("""
select username
from reviews
where user_rating between 7 and 10
group by username
having count(username) >= 10
order by count(username) desc
""")

reviewers = c.fetchall()

In [230]:
# Get positive reviews from database
c.execute("SELECT movie_id, username FROM reviews WHERE user_rating BETWEEN 7 and 10")
result = c.fetchall()

# create reviews dataframe
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,0233469,elmonose
1,0234105,scott_thai
2,0233469,JACKAL-22
3,0233469,o0supergeek0o
4,0233469,steveefford


In [231]:
# create reviewers dataframe
df_reviewers = pd.DataFrame(reviewers, columns = ['userid'])

In [232]:
# merge to get only the IDs relevant to training
df = df.merge(df_reviewers, how='inner', on='userid')
df.shape

(817687, 2)

In [22]:
# ! sudo su
# ! yum update -y
# ! yum -y install python-pip
# ! python -V

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Python 3.6.5 :: Anaconda, Inc.


In [23]:
# ! which pip

/home/ec2-user/anaconda3/envs/python3/bin/pip


# Install gensim

In [234]:
! python -m pip install tqdm
# ! python -c 'import tqdm'
# ! python -m pip install gensim

    100% |████████████████████████████████| 61kB 25.4MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [235]:
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [238]:
# list to capture watch history of the users
watched_train = []

# populate the list with the movie codes
for i in tqdm(reviewers):
    temp = df[df["userid"] == i[0]]["movieid"].tolist()
    watched_train.append(temp)
    
len(watched_train)

100%|██████████| 17812/17812 [41:02<00:00,  7.23it/s]


17812

In [239]:
# save the model for later
import pickle
pickle.dump(watched_train, open('watched_train_2.6.sav', 'wb'))

In [166]:
# #save the model in protocol 2 so it can be opened in python 2.7
# import pickle
# temp = pickle.load(open('watched_train.sav', 'rb'))
# pickle.dump(temp, open('watched_train.sav', 'wb'), protocol=2)

### Train the Model

**Important:** The previous model was trained on movie IDs that were inside lists of length 1, with watch histories being lists of lists.

This model eschews the inner lists. Each watch history is simply a list of strings.

In [237]:
import pickle
watched_train = pickle.load(open('watched_train.sav', 'rb'))

In [240]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watched_train, progress_per=200)

model.train(watched_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(7410030, 8176870)

In [241]:
# save word2vec model
model.save("w2v_mistakenot_v2.6.model")

### Test the model

In [6]:
!pip install gensim

    100% |████████████████████████████████| 24.2MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 33.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
You are using pip version 10.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
# load model
import gensim
model = gensim.models.Word2Vec.load("w2v_mistakenot_v2.6.model")

In [242]:
# prunes the model, making it faster but unable to train any more.
model.init_sims(replace=True)

In [243]:
print(model)

Word2Vec(vocab=23003, size=100, alpha=0.03)


In [244]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(23003, 100)

In [245]:
import pandas as pd
ratings = pd.read_csv('ratings.csv') # import my Letterboxd ratings
ratings.head()
ratings = ratings.dropna(axis=0, subset=['Rating', 'Name', 'Year'])

# set thresholds for minimum "good" rating and maximum "bad rating"
good_threshold = 4
bad_threshold = 3
good = ratings[ratings['Rating'] >= good_threshold]
bad = ratings[ratings['Rating'] <= bad_threshold]

# good_from_4 = ratings[ratings['Rating'] >= good_threshold]
good_from_3p5 = ratings[ratings['Rating'] >= 3.5]

# get watchlist for scoring the model
watchlist = pd.read_csv('watchlist.csv').dropna(axis=0, subset=['Name', 'Year'])

# get watch history sans ratings
full_history = pd.read_csv('watched.csv').dropna(axis=0, subset=['Name', 'Year'])

def df_to_id_list(df):
    """
    Input: dataframe of movies from my ratings.csv Letterboxd export
    
    Output: List of the IDs for those movies, ready for inferencing.
    """
    ids = []
    names = df.Name.tolist()
    names = [x.replace("'", "") for x in names]
    years = df.Year.tolist()
    years = [int(year) for year in years]
    info = list(zip(names, years))
    for i, j in info:
        try:
            c.execute(f"""
                SELECT movie_id
                FROM movies
                WHERE primary_title LIKE '{i}' AND start_year = {j}
                ORDER BY runtime_minutes DESC
                LIMIT 1""")
            id = c.fetchone()[0]
            ids.append(id)
        except:
            continue
    return ids



# get ids from all lists, Dr. Seuss style
good_list = df_to_id_list(good)
bad_list = df_to_id_list(bad)
val_list = df_to_id_list(watchlist)
hist_list = df_to_id_list(full_history)
good_3p5_list = df_to_id_list(good_from_3p5)
good_list_long = good_list[:int(len(good_list)/2)]
good_list_short = good_list[int(len(good_list)/2):]

print(len(good_list), len(bad_list), len(val_list), len(hist_list), len(good_3p5_list))

375 179 1002 1086 515


In [246]:
val_list = df_to_id_list(watchlist)

In [247]:
class ScoringService(object):
    model = None                # Where we keep the model when it's loaded
    
    @classmethod
    def get_model(cls):
        """Get the model object for this instance, loading it if it's not already loaded."""
        if cls.model == None:
            # load the gensim model
            w2v_model = gensim.models.Word2Vec.load("w2v_mistakenot.model")
            # keep only the normalized vectors.
            # This saves memory but makes the model untrainable (read-only).
            w2v_model.init_sims(replace=True)
            # with open(os.path.join(model_path, 'decision-tree-model.pkl'), 'r') as inp:
            #     cls.model = pickle.load(inp)
            cls.model = w2v_model
        return cls.model

    @classmethod
    def predict(cls, input, bad_movies=[], n=20, harshness=1, rec_movies=True, show_vibes=False, scoring=False):
        """Returns a list of recommendations and useful metadata, given a pretrained 
        word2vec model and a list of movies.

        Args:
            cls (.model object): The pretrained word2vec model.
            
            input (list of strings): The list of movies that the user likes.
            
            bad_movies (list of strings): The list of movies that the user dislikes.
            
            n (int): The number of recommendations to return.
            
        Output: A list of tuples: Title, Year, IMDb URL, Similarity score.
        """
        
        # get pretrained model
        clf = cls.get_model()
        
        # list for storing duplicates
        dupes = []

        def _aggregate_vectors(movies):
            """Gets the vector average of a list of movies."""
            movie_vec = []
            for i in movies:
                try:
                    movie_vec.append(clf[i]) # get the vector for each movie
                except KeyError:
                    continue
            return np.mean(movie_vec, axis=0)

        def _similar_movies(v, bad_movies=[], n = 10):
            """Takes aggregated vector of good movies, 
            and optionally, a list of disliked movies.
            Subtracts disliked movies.
            Returns most similar movies for the input vector
            
            n: number of recommendations to return."""
            if bad_movies:
                v = _remove_dislikes(bad_movies, v, input=input, harshness=harshness)
            return clf.similar_by_vector(v, topn= n+1)[1:]
            
        def _remove_dupes(recs, input, bad_movies):
            """remove any recommended IDs that were in the input list"""
            all_seen = input + bad_movies
            # if there is a full watch history list, use it to remove dupes
            if hist_list:
                all_seen = list(set(all_seen+hist_list))
            nonlocal dupes
            dupes = [x for x in recs if x[0] in all_seen]
            return [x for x in recs if x[0] not in all_seen]

        def _get_info(id):
            """Takes an id string and returns the movie info with a url."""
            try:
                c.execute(f"""
                select m.primary_title, m.start_year, r.average_rating, r.num_votes
                from movies m
                join ratings r on m.movie_id = r.movie_id
                where m.movie_id = '{id[0]}'""")
            except:
                return tuple([f"Movie title unknown. ID:{id[0]}", None, None, None, None, None])

            t = c.fetchone()
            if t:
                title = tuple([t[0], t[1], f"https://www.imdb.com/title/tt{id[0]}/", t[2], t[3], id[1]])
                return title
            else:
                return tuple([f"Movie title unknown. ID:{id[0]}", None, None, None, None, None])
            
                
        def _remove_dislikes(bad_movies, good_movies_vec, input=1, harshness=1):
            """Takes a list of movies that the user dislikes. 
            Their embeddings are averaged,
            and subtracted from the input."""
            bad_vec = _aggregate_vectors(bad_movies)
            bad_vec = bad_vec / harshness
            return good_movies_vec - bad_vec
        
        def _score_model(recs, val_list):
            ids = [x[0] for x in recs]
            return len(list(set(ids) & set(val_list)))

        recs = _aggregate_vectors(input)
        recs = _similar_movies(recs, bad_movies, n=n)
        recs = _remove_dupes(recs, input, bad_movies)
        formatted_recs = [_get_info(x) for x in recs]
        
        if scoring and val_list:
            print(f"The model recommended {_score_model(recs, val_list)} movies that were on the watchlist!\n")
            print(f"\t\t Average Rating: {sum([i[3] for i in formatted_recs if i[3] is not None])/len(formatted_recs)}\n")
        if show_vibes:
            print("Movies you're likely to vibe with people on: \n")
            for x in dupes:
                print(_get_info(x))
            print('\n')
        if rec_movies:
            return formatted_recs

In [248]:
# test cases

# A list of some Coen Bros movies.
coen_bros = ['116282', '2042568', '1019452', 
             '1403865', '190590', '138524', 
             '335245', '477348', '887883', '101410']

# Data scientist's recent watches.
cooper_recent = ['0053285', '0038650', '0046022', 
                 '4520988', '1605783', '6751668', 
                 '0083791', '0115685', '0051459', 
                 '8772262', '0061184', '0041959',
                 '7775622']

# dirkh public letterboxd recent watches.
dirkh = ['7975244', '8106534', '1489887', 
         '1302006', '7286456', '6751668', 
         '8364368', '2283362', '6146586', 
         '2194499', '7131622', '6857112']

# Marvin watches
marvin = ['7286456', '0816692', '2543164', '2935510', 
          '2798920', '0468569', '5013056', '1375666', 
          '3659388', '0470752', '0266915', '0092675', 
          '0137523', '0133093', '1285016']  

# Gabe watches
gabe = ['6292852','0816692','2737304','3748528',
        '3065204','4154796','1536537','1825683',
        '1375666','8236336','2488496','1772341',
        '0317705','6857112','5052448']

# Eric watches
eric = ['2974050','1595842','0118539','0093405',
        '3216920','1256535','5612742','3120314',
        '1893371','0046248','0058548','0199481',
        '2296777','0071198','0077834']

chuckie = ['4263482',
'0084787',
'3286052',
'5715874',
'1172994',
'4805316',
'3139756',
'8772262',
'7784604',
'1034415',]

harlan = ['1065073','5052448','0470752','5688932','1853728','1596363','0432283','6412452','4633694','9495224','0443453','0063823',
          '0066921','0405296','1130884','1179933','0120630','0268126','0137523','0374900','8772262','0116996','0107290','7339248']

ryan = ['0166924','2866360','0050825','2798920','3416742','0060827','1817273','0338013','0482571','5715874','2316411','4550098']

karyn = ['4425200','0464141','1465522','0093779','0099810','0076759','3748528','6763664','0317740','2798920','0096283','0258463','0118799','0058092','0107290','0045152','0106364']

richard = ['0074119','0064115','0070735','0080474','0061512','0067774','0057115','0070511','0081283',
           '0065126','0068421','0078227','0079100','0078966','0081696','0082085','0072431','0075784',
           '0093640','0098051','0094226','0097576','0099810','0081633','0080761','0077975','0085244','0095159','0101969']

joe = ['6335734','0291350','0113568','0208502','0169858','0095327','0097814','0983213','0094625','7089878']

lena = ['1990314','3236120','1816518','0241527','0097757','0268978','0467406','2543164','2245084','3741834']

wade = ['0118665','0270846','0288441','2287250','2287238','8668804','9448868','1702443','1608290','5519340']

In [249]:
s = ScoringService()

In [250]:
# define many parameter sets

# possible good inputs: cooper_recent (13 most recent), good_list(my 4-5 star ratings), good_3p5_list (my 3.5-5 star ratings)
# possible bad_movies: [] (not removing dislikes), bad_list(my ratings 3 and below), karyn (individual whose taste markedly differs from mine)
# possible harshness: 1, 2, 3 (harshness 1 strongly removes disliked movies, while harshness 3 removes with a third of the strength)

params = [
    (cooper_recent, [], 1), #1
    (cooper_recent, bad_list, 1),
    (cooper_recent, bad_list, 2),
    (cooper_recent, bad_list, 3),
    (good_list, [], 1), #5
    (good_list, bad_list, 1),
    (good_list, bad_list, 2),
    (good_list, bad_list, 3),
    (good_3p5_list, [], 1),
    (good_3p5_list, bad_list, 1), #10
    (good_3p5_list, bad_list, 2),
    (good_3p5_list, bad_list, 3),
    (cooper_recent, karyn, 1),
    (good_list, karyn, 1),
    (good_3p5_list, karyn, 1), #15
]

count = 0
# Test all parameter sets
for i, j, k in params:
    count+=1
    print(count, "\t")
    s.predict(input=i, bad_movies=j, n=100, harshness=k, rec_movies=False, show_vibes=True, scoring=True)

1 	
The model recommended 12 movies that were on the watchlist!

		 Average Rating: 7.252127659574468

Movies you're likely to vibe with people on: 

('Free Solo', 2018, 'https://www.imdb.com/title/tt7775622/', 8.2, 43185, 0.7211458683013916)
('Isle of Dogs', 2018, 'https://www.imdb.com/title/tt5104604/', 7.9, 120894, 0.7097740173339844)
('Spider-Man: Into the Spider-Verse', 2018, 'https://www.imdb.com/title/tt4633694/', 8.4, 290411, 0.6956536769866943)
('Frozen II', 2019, 'https://www.imdb.com/title/tt4520988/', 7.2, 59468, 0.6918184757232666)
('Us', 2019, 'https://www.imdb.com/title/tt6857112/', 6.9, 177279, 0.6857053637504578)
('Parasite', 2019, 'https://www.imdb.com/title/tt6751668/', 8.6, 157194, 0.6763735413551331)


2 	
The model recommended 14 movies that were on the watchlist!

		 Average Rating: 7.208163265306121

Movies you're likely to vibe with people on: 

('The Earrings of Madame De...', 1953, 'https://www.imdb.com/title/tt0046022/', 8.0, 8182, 0.3684726357460022)
('Para

In [251]:
#Compare the unique recs of settings 7 and 11 from above.
#The param difference is that setting 7 uses movies rated 4 and above,
#while setting 11 uses movies rated 3.5 and above.

p4 = s.predict(input=good_list, bad_movies=bad_list, n=200, harshness=2, scoring=True)
p35 = s.predict(input=good_3p5_list, bad_movies=bad_list, n=200, harshness=2, scoring=True)

print(len(p4), len(p35))

p4_unique = [x for x in p4 if x[0] not in [x[0] for x in p35]]
p35_unique = [x for x in p35 if x[0] not in [x[0] for x in p4]]

print(len(p4_unique), len(p35_unique))


The model recommended 42 movies that were on the watchlist!

		 Average Rating: 7.5668789808917225

The model recommended 37 movies that were on the watchlist!

		 Average Rating: 7.53533333333333

157 150
44 37


In [252]:
for i in p4_unique:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}")

A Moment of Innocence	1996
	https://www.imdb.com/title/tt0117214/
East Palace, West Palace	1996
	https://www.imdb.com/title/tt0119007/
Nayak: The Hero	1966
	https://www.imdb.com/title/tt0060742/
Conversation Piece	1974
	https://www.imdb.com/title/tt0071585/
Fast, Cheap & Out of Control	1997
	https://www.imdb.com/title/tt0119107/
Weekend	1967
	https://www.imdb.com/title/tt0062480/
The Mirror	1975
	https://www.imdb.com/title/tt0072443/
The Long Day Closes	1992
	https://www.imdb.com/title/tt0104753/
Short Cuts	1993
	https://www.imdb.com/title/tt0108122/
On the Waterfront	1954
	https://www.imdb.com/title/tt0047296/
Cries & Whispers	1972
	https://www.imdb.com/title/tt0069467/
Teorema	1968
	https://www.imdb.com/title/tt0063678/
Signs of Life	1968
	https://www.imdb.com/title/tt0063218/
Belle de Jour	1967
	https://www.imdb.com/title/tt0061395/
The Travelling Players	1975
	https://www.imdb.com/title/tt0073800/
Ivan's Childhood	1962
	https://www.imdb.com/title/tt0056111/
The Story of Qiu Ju	1992

In [212]:
for i in p35_unique:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}")

Once Upon a Time in High School: The Spirit of Jeet Kune Do	2004
	https://www.imdb.com/title/tt0390205/
Chillar Party	2011
	https://www.imdb.com/title/tt1841542/
Angel's Egg	1985
	https://www.imdb.com/title/tt0208502/
Unacknowledged	2017
	https://www.imdb.com/title/tt6400614/
Lootera	2013
	https://www.imdb.com/title/tt2224317/
Awe!	2018
	https://www.imdb.com/title/tt7797658/
Vada Chennai	2018
	https://www.imdb.com/title/tt5959980/
Aruvi	2016
	https://www.imdb.com/title/tt5867800/
Big Fish & Begonia	2016
	https://www.imdb.com/title/tt1920885/
Thithi	2015
	https://www.imdb.com/title/tt4881362/
Boy and the World	2013
	https://www.imdb.com/title/tt3183630/
Neon Genesis Evangelion: The End of Evangelion	1997
	https://www.imdb.com/title/tt0169858/
That Girl in Yellow Boots	2010
	https://www.imdb.com/title/tt1580704/
The Breath	2009
	https://www.imdb.com/title/tt1171701/
The Stupids	1996
	https://www.imdb.com/title/tt0117768/
To the Wonder	2012
	https://www.imdb.com/title/tt1595656/
The Meyer

In [253]:
prediction = s.predict(input=good_list, bad_movies=bad_list, n=200, harshness=2)
for i in prediction:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}\n\tAvg. Rating: {i[3]}\n\t# Votes: {i[4]}\n\tSimilarity: {i[5]}\n\n")

The Thin Blue Line	1988
	https://www.imdb.com/title/tt0096257/
	Avg. Rating: 8.0
	# Votes: 20413
	Similarity: 0.688605785369873


Mommy	2014
	https://www.imdb.com/title/tt3612616/
	Avg. Rating: 8.1
	# Votes: 45282
	Similarity: 0.6870128512382507


A Short Film About Killing	1988
	https://www.imdb.com/title/tt0095468/
	Avg. Rating: 8.1
	# Votes: 15515
	Similarity: 0.6861892342567444


A Short Film About Love	1988
	https://www.imdb.com/title/tt0095467/
	Avg. Rating: 8.2
	# Votes: 17487
	Similarity: 0.6853230595588684


Mood Indigo	2013
	https://www.imdb.com/title/tt2027140/
	Avg. Rating: 6.5
	# Votes: 14541
	Similarity: 0.6825548410415649


Trainspotting	1996
	https://www.imdb.com/title/tt0117951/
	Avg. Rating: 8.1
	# Votes: 606132
	Similarity: 0.6801393032073975


The Firemen's Ball	1967
	https://www.imdb.com/title/tt0061781/
	Avg. Rating: 7.5
	# Votes: 8519
	Similarity: 0.6788325309753418


71 Fragments of a Chronology of Chance	1994
	https://www.imdb.com/title/tt0109020/
	Avg. Rating:

In [257]:
'6615648' in full_history.Name.tolist()

False

In [258]:
len(full_history.Name.tolist())

1420